In [51]:
from Preprocess import Process
import pickle
import os
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD, Adagrad, Adadelta, RMSprop, Adam
from keras.models import model_from_json
import datetime
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras import datasets, layers, models
import keras
from sklearn.model_selection import train_test_split

In [15]:
directory="./data/BanglaData/"
length=30
checkpoint_path = "cer_result_100_new.ckpt"
base_learning_rate=0.01
epoch=50
batch_size=64

# Training Test Split

In [52]:
process= Process(directory,length)
sequences,mapping=process.encode_seq()
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)

cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,
                                                 save_weights_only=True,
                                                 verbose=1)

# Moldel Definition

In [49]:
model = Sequential()
model.add(Embedding(vocab, 50,embeddings_initializer='uniform', input_length=length, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.2))
model.add(Dense(vocab, activation='softmax'))

learning_rate = 0.1

# Set Optimizer
opt = SGD(lr=1e-2, decay=1e-2/100)
#opt = Adam(lr=1e-2)

# Compile model
#model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer=opt)
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')

model.summary()
model.save('my_model.h5') 

In [50]:
history = model.fit(X_tr, y_tr,
                batch_size=batch_size,
                epochs=epoch,
                verbose=1,
                initial_epoch=0,
                validation_data=(X_val, y_val),
                     callbacks = [cp_callback,tensorboard_callback])

# Test Script

In [31]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

# Charecter Level Text Generation Using Our Trained Model

In [48]:
inp="ভাষা"
generate_seq(model, mapping, 30, inp,14)

'ভাষার সাথে সম্পর্ক'